__Tutorial 03 - Introduction to ChipWhisperer__

# Introduction

## Side-Channel Analysis (SCA) Attacks

As computer hardware devices perform various computations, their physical properties can be measured and used to identify what "functions/operations" are being performed and even the contents of the underlying "data." These physical properties are referred to as side-channel leakages, and come in many forms. For example, as the circuit is performing the operations, the effects can be seen on the power consumption on the chip, as well as in the electromagnetic and thermal emanations measured externally. The usage of side-channel leakages to break into a system is known as "Side-Channel Analysis (SCA) Attacks."

In this tutorial, we will use the power consumption from running scripts (firmware) to determine when has the code terminated its execution. In other words, we will use power SCA to facilitate on finding information leakage from the system.

## ChipWhisperer (CW)

The ChipWhisperer (CW) Nano (see Fig. 1) is a low cost platform for performing power Side-Channel Analysis (SCA) attacks in training and educational environments. This device has 2 main section: TARGET and CAPTURE.

* __TARGET Section:__ This part holds a basic micro-controller (µController) that will run the "Victim" code we want to attack. 
* __CAPTURE Section:__ This contains all the circuit necessary to collect the power traces from the TARGET section. This behaves as an oscilloscope (or Scope) that only collects power traces from the "Victim µController." This section has another µController that performs all the computations and interface (i.e., input/output connections) for the Scope and some memory to save all the power samples collected before sending to the PC.
Since the power values are "Analog" signals, an "Analog-to-Digital Converter" (ADC) will convert this signal to "Digital" signal (i.e., binary) so it can be saved and sent to the PC.

Both the CAPTURE and TARGET section can communicate with the PC using the same USB-B (or micro-USB) port.

<span class="mark">![fig_cw](../Figures/fig_cw.png)
Fig. 1: Illustration of the ChipWhisperer-Nano and its main important parts.</span>

We will use the CW-Nano to run the victim script on the CW's "target" and collect its power consumption traces with the integrated CW's "scope" on the capture section.

To facilitate, we will use this Jupyter Notebook to connect and interact with the CW using simple/easy python code. We provided with all the necessary code and instructions, just follow the steps on the next sections.

# Create a Firmware Script

Before starting, we need to make a firmware (victim) script in C language to be running on the CW's target.

* Under`cw-base-setup/simpleserial-base` directory, make a copy of the `simpleserial-base` files (`simpleserial-base.c` and the `makefile`) and move them into your `PHS-Tutorial-03-CW/Firmware` directory.

* Under `PHS-Tutorial-03-CW/Firmware` directory, rename the `simpleserial-base.c` copy to `tutorial-cw.c`, and update the `makefile` accordingly:
    * lines 33 and 37 of the `makefile` with the updated file name.
    * because the files moved location, lines 45 - 49 replace with:

* Open the `tutorial-cw.c` file and remove the following functions (not needed): `get_key`, `reset`, and `aes`. Also, remove `#if SS_VER == SS_VER_2_1` and `#endif` around the `aes` function. You should only have the 4 `#include` lines and 2 functions: `get_pt` and `main`.

* Using the function `get_pt` as a template, create two new functions named: `read_and_send` and `get_sum`.

## read_and_send function

Let's start with `read_and_send` function. This function will just read an array of 16 bytes (i.e., unsigned integers from 0 to 255) and then send it back. Below is a screenshot of how this function should look like. Please, write this code for your `read_and_send` function:

<span class="mark">![read_and_send_code](../Figures/read_and_send_code.png)
Fig. 2: "read_and_send" function code (in C) parts.</span>

We will test this script later in sections 3-5.

__What does this code mean?__

* First, it will accept an array of 16 bytes (8-bits unsigned int).
* Then, indicate the start (`trigger_high()`) and end (`trigger_low()`) of the data collection. In this case, there is nothing to collect.
* Finally, sends back (using simpleserial) the same array `pt` to the user. You will be able to read this from the CW.

## get_sum function

Now, let's move to `get_sum` function. This function will just read an array of 16 bytes (i.e., unsigned integers from 0 to 255), then perform a sum across all 16 elements of the input array, and return the final value. Below is a screenshot of how this function should look like. Please, write this code for your `get_sum`:

<span class="mark">![get_sum_code](../Figures/get_sum_code.png)
Fig. 3: "get_sum_code" function code (in C) parts.</span>

We will test this script later in sections 3-5.

__What does this code mean?__

* First, it will accept an array of 16 bytes (8-bits unsigned int).
* Second, initializes some variables:
    * `i` for the loop iterations, 
    * `sum` to keep track of the sum, and 
    * `result` to convert the `sum` to a 16 bytes array needed for sending to the user through simpleserial.
* Third, indicate the start (`trigger_high()`) and end (`trigger_low()`) of the data collection. In this case, there is nothing to collect.
    * In between these indicators, we wrote the victim code we want to capture its power trace. In this case, the loop doing the `sum` across all 16 bytes from the input array.
* Fourth, convert the `sum` to a byte array in big endian. Meading, we need to extract every least significant 8 bits from `sum` and same in the `result` array (right to left).

* Finally, send (using simpleserial) the `result` array to the user. You will be able to read this from the CW.

## main function

Now, let's move to `main` function. This function will setup and initialize the CW, UART, scope triggers, and simpleserial. Then, it will associate your previous functions to a command (i.e., a character) to the simpleserial. Lastly, it will infinitely loop requesting and waiting for a command from the user to do something. Below is a screenshot of how this function should look like. Please, write this code for your `main` function:

<span class="mark">![main_code](../Figures/main_code.png)
Fig. 2: "get_sum_code" function code (in C) parts.</span>

We will test this script later in sections 3-5.

__What does this code mean?__

* First, initializes and setsup the CW and simpleserial (to communicate with the CW).
* Then, associate each of your functions to a command (1 character) and add this to the simpleserial.
    * Syntax: `simpleserial_addcmd(cmd, 16, funct_name);`
    * `cmd`: 1 character command (different for each function), and
    * `funct_name`: name of the function (<span class="burk">NOT</span> as a string).
* Finally, wait for a command from the user for the respective function: 'a' for `read_and_send` or 'b' for `get_sum`.

# Setup

## Importing Python Libraries & Setup ChipWhisperer

Now that the firmware is done, let's get to the Python part:

Begin by importing the required libraries, connecting to the scope, and applying the default setup:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import chipwhisperer as cw
import numpy as np
import scipy.stats
from tqdm import tnrange


# This is an API for your CW's scope (in the CAPTURE section)
scope = cw.scope()

# This is an API for your CW's target (in the TARGET section)
target = cw.target(scope, cw.targets.SimpleSerial)

# Sets the scope's default settings
scope.default_setup()

# Cap the max num of power trace samples to collect
scope.adc.samples = 500

# Prints the scope settings
print(scope)

print("✔️ OK to continue!")

Next, compile the firmware (the C code you wrote):

In [ ]:
%%bash
cd ../Firmware
make PLATFORM=CWNANO CRYPTO_TARGET=NONE

Program the microcontroller with the "tutorial-cw-CWNANO" hex file:

In [ ]:
prog = cw.programmers.STM32FProgrammer
fw_path = '../Firmware/tutorial-cw-CWNANO.hex'
cw.program_target(scope, prog, fw_path)

print("✔️ OK to continue!")

## Function to Reset Target

This function allow you to reset the target from your code:

In [ ]:
import time
def reset_target(): 
    target.flush()
    scope.io.nrst = 'low'
    time.sleep(0.2)
    scope.io.nrst = 'high'
    time.sleep(0.01)

Call this two function here:

In [ ]:
reset_target()
print("✔️ OK to continue!")

# Send/Read Data

Now that you have setup the CW, then send and read some data from the target.

## read_and_send function

Test your `read_and_send` function (command = 'a'). Sends 16 random bytes (hardcoded).
Then, verify the results using python.

In [ ]:
# reset target and read start-up text
reset_target()

# example data list to send
d = [3, 5, 26, 22, 200, 12, 45, 23, 87, 43, 97, 157, 138, 234, 27, 2]

# converts list to bytearray
b = bytearray(d)

# send the bytearray to the CW function with command 'a'
target.simpleserial_write('a', b)

# read data from CW
r = target.simpleserial_read('r', 16)

# print data response:
print(r)

# convert data respose from bytearray to list of int, then print
result = list(r)
print(result)

# verify if response value match expectation
assert result == d
print("✔️ OK to continue!")

## get_sum function

Test your `get_sum` function (command = 'b'). Uses the same bytearray `b` with the same data from before.
Then, verify the results using python.

In [ ]:
# reset target and read start-up text
reset_target()

# send the bytearray to the CW function with command 'b'
target.simpleserial_write('b', b)

# read data from CW
r = target.simpleserial_read('r', 16)

# print data response:
print(r)

# convert data respose from bytearray to int, then print
result = int.from_bytes(r, "big")
print(result)

# verify if response value match expectation
assert result == sum(d)
print("✔️ OK to continue!")

# Record & Plot Traces

Now that you can communicate with your system and the firmware are correct, our next goal is to get a power trace while the target is running. To do this, you'll arm the scope just before you send the data, then record the trace as you did in the tutorial.

Hint: it is similar to sending/reading data as before, but instead of reading data from the CW, you are capturing power traces.

## read_and_send function

Get power traces from your `read_and_send` function (command = 'a'). Use the same bytearray `b` with the same data from before.

In [ ]:
# reset target and read start-up text
reset_target()

# arm the CW scope to be ready to capture power traces
scope.arm()

# send the bytearray to the CW function with command 'b'
target.simpleserial_write('a', b)

# start the scope to capture power traces
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')

# get the power trace from the CW
trace1 = scope.get_last_trace()

# print number of samples:
print(len(trace1))

# print some sample power traces (only first 50)
print(trace1[:50])

# verify if the trace is collected correctly
assert type(trace1) == np.ndarray
assert len(trace1)  == 500
print("✔️ OK to continue!")

Then, plot the power traces:

In [ ]:
# makes new empty plot
plt.figure(figsize=(5.5, 3.5), constrained_layout=True)
# figsize: (width, height) in inches.
# constrained_layout: makes sures the plot fits tight within the image boundaries.

# plots the data
plt.plot(trace1, color="b")

# format your plots
plt.title("Title")            # adds title
plt.xlabel("Sample Number")  # adds x-axis label
plt.ylabel("Power")          # adds y-axis label
# ... and more ...

# saves the plot
plt.savefig("../Figures/fig-name1.pdf")

# show the plot on your screen
plt.show()

## get_sum function

Get power traces from your `get_sum` function (command = 'a'). Use the same bytearray `b` with the same data from before.

In [ ]:
# reset target and read start-up text
reset_target()

# arm the CW scope to be ready to capture power traces
scope.arm()

# send the bytearray to the CW function with command 'b'
target.simpleserial_write('b', b)

# start the scope to capture power traces
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')

# get the power trace from the CW
trace2 = scope.get_last_trace()

# print number of samples:
print(len(trace2))

# print some sample power traces (only first 50)
print(trace2[:50])

# verify if the trace is collected correctly
assert type(trace2) == np.ndarray
assert len(trace2)  == 500
print("✔️ OK to continue!")

Then, plot the power traces:

In [ ]:
# makes new empty plot
plt.figure(figsize=(5.5, 3.5), constrained_layout=True)
# figsize: (width, height) in inches.
# constrained_layout: makes sures the plot fits tight within the image boundaries.

# plots the data
plt.plot(trace2, color="b")

# format your plots
plt.title("Title")            # adds title
plt.xlabel("Sample Number")  # adds x-axis label
plt.ylabel("Power")          # adds y-axis label
# ... and more ...

# saves the plot
plt.savefig("../Figures/fig-name2.pdf")

# show the plot on your screen
plt.show()

## compare function

Let's plot both traces on the same figure and compare.

In [ ]:
# makes new empty plot
plt.figure(figsize=(5.5, 3.5), constrained_layout=True)
# figsize: (width, height) in inches.
# constrained_layout: makes sures the plot fits tight within the image boundaries.

# plots the data
plt.plot(trace2[:250], color="b", label="get_sum")
plt.plot(trace1[:250], color="r", label="rean_and_send")

# format your plots
plt.title("Title")            # adds title
plt.xlabel("Sample Number")  # adds x-axis label
plt.ylabel("Power")          # adds y-axis label
plt.legend()
# ... and more ...

# saves the plot
plt.savefig("../Figures/fig-name3.pdf")

# show the plot on your screen
plt.show()

Notice that `get_sum` have particular power spikes at each iteration of the sum. 16 spikes for 16 iterations.

However, `read_and_send` does not have much of relevant power spikes. The random power consumption is mostly random noise and background operations unrelated to this function.

# Diconnect Scope/Target

After you are done, always diconnect the CW's scope and target:

In [ ]:
scope.dis()
target.dis()